<a href="https://colab.research.google.com/github/KimJisanER/Dacon/blob/main/1_2_penguin_MassReg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing
import seaborn as sns

In [166]:
dataset_path='/content/drive/MyDrive/penguin/'

train_dataset = pd.read_csv(dataset_path+'train.csv').drop(['id'],axis=1)
test_dataset = pd.read_csv(dataset_path+'test.csv').drop(['id'],axis=1)

In [167]:
print(train_dataset.isnull().sum())
print(test_dataset.isnull().sum())

Species                0
Island                 0
Clutch Completion      0
Culmen Length (mm)     0
Culmen Depth (mm)      0
Flipper Length (mm)    0
Sex                    3
Delta 15 N (o/oo)      3
Delta 13 C (o/oo)      3
Body Mass (g)          0
dtype: int64
Species                0
Island                 0
Clutch Completion      0
Culmen Length (mm)     0
Culmen Depth (mm)      0
Flipper Length (mm)    0
Sex                    6
Delta 15 N (o/oo)      9
Delta 13 C (o/oo)      8
dtype: int64


In [168]:
path='/content/drive/MyDrive/penguin/'

train=pd.read_csv(path+'train.csv')
test=pd.read_csv(path+'test.csv')

X_train=train.iloc[:,1:10]
y_train=train.loc[:,'Body Mass (g)']
test_id=test.iloc[:,0]
X_test=test.iloc[:,1:]

X_train['Delta 15 N (o/oo)']=X_train['Delta 15 N (o/oo)'].fillna(X_train['Delta 15 N (o/oo)'].mean())
X_test['Delta 15 N (o/oo)']=X_test['Delta 15 N (o/oo)'].fillna(X_test['Delta 15 N (o/oo)'].mean())

X_train['Delta 13 C (o/oo)']=X_train['Delta 13 C (o/oo)'].fillna(X_train['Delta 13 C (o/oo)'].mean())
X_test['Delta 13 C (o/oo)']=X_test['Delta 13 C (o/oo)'].fillna(X_test['Delta 13 C (o/oo)'].mean())

label=[]

for i in X_train.columns:
  if X_train.dtypes[i] == 'object':
    label.append(i)

from sklearn.preprocessing import LabelEncoder
X_train.loc[:,label]=X_train.loc[:,label].apply(LabelEncoder().fit_transform)
X_test.loc[:,label]=X_test.loc[:,label].apply(LabelEncoder().fit_transform)


from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)


from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(max_depth=5, n_estimators=200, min_samples_leaf=8, min_samples_split=8)
model.fit(X_train,y_train)

from sklearn.linear_model import Lasso
model_la=Lasso(alpha=0.2)
model_la.fit(X_train,y_train)

from sklearn.linear_model import LinearRegression
model_lr=LinearRegression()
model_lr.fit(X_train,y_train)

import xgboost as xgb
from xgboost import XGBRegressor
model_xgb=XGBRegressor(max_depth=2)
model_xgb.fit(X_train,y_train)

pred=model.predict(X_test)
pred2=model_la.predict(X_test)
pred3=model_lr.predict(X_test)
pred4=model_xgb.predict(X_test)

pred_pre=0.25*pred+0.25*pred2+0.25*pred3+0.25*pred4
pred_pre=pd.DataFrame(pred_pre)
pred_pre.columns=['Body Mass (g)']
print(pred_pre)

[13:51:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
     Body Mass (g)
0      4420.073025
1      5568.529158
2      3640.425762
3      3421.336572
4      3472.991308
..             ...
223    4188.594756
224    4765.359057
225    4598.446465
226    3965.292552
227    4068.702435

[228 rows x 1 columns]


In [169]:
test_dataset=pd.concat([test_dataset, pred_pre], axis=1)
print(test_dataset)

                                       Species  ... Body Mass (g)
0    Chinstrap penguin (Pygoscelis antarctica)  ...   4420.073025
1            Gentoo penguin (Pygoscelis papua)  ...   5568.529158
2          Adelie Penguin (Pygoscelis adeliae)  ...   3640.425762
3    Chinstrap penguin (Pygoscelis antarctica)  ...   3421.336572
4          Adelie Penguin (Pygoscelis adeliae)  ...   3472.991308
..                                         ...  ...           ...
223  Chinstrap penguin (Pygoscelis antarctica)  ...   4188.594756
224          Gentoo penguin (Pygoscelis papua)  ...   4765.359057
225          Gentoo penguin (Pygoscelis papua)  ...   4598.446465
226  Chinstrap penguin (Pygoscelis antarctica)  ...   3965.292552
227        Adelie Penguin (Pygoscelis adeliae)  ...   4068.702435

[228 rows x 10 columns]


In [170]:
train_dataset['Sex']=np.where(train_dataset['Sex'].values=='MALE',1,np.where(train_dataset['Sex'].values=='FEMALE',0,np.nan))
test_dataset['Sex']=np.where(test_dataset['Sex'].values=='MALE',1,np.where(test_dataset['Sex'].values=='FEMALE',0,np.nan))
train_dataset['Clutch Completion']=np.where(train_dataset['Clutch Completion'].values=='Yes',1,0)
test_dataset['Clutch Completion']=np.where(test_dataset['Clutch Completion'].values=='Yes',1,0)
train = pd.concat([train_dataset,pd.get_dummies(train_dataset[['Island','Species']])],axis=1)
test = pd.concat([test_dataset,pd.get_dummies(test_dataset[['Island','Species']])],axis=1)
train = train.drop(['Island','Species'],axis=1)
test = test.drop(['Island','Species'],axis=1)

In [171]:
all=pd.concat([train,test],axis=0)
all.to_csv('all.csv',index=False)
print(all)
print(all.isnull().sum())
all=all.dropna(axis=0)
print(all.isnull().sum())

     Clutch Completion  ...  Species_Gentoo penguin (Pygoscelis papua)
0                    1  ...                                          1
1                    0  ...                                          0
2                    1  ...                                          1
3                    1  ...                                          1
4                    0  ...                                          1
..                 ...  ...                                        ...
223                  1  ...                                          0
224                  1  ...                                          1
225                  1  ...                                          1
226                  1  ...                                          0
227                  1  ...                                          0

[342 rows x 14 columns]
Clutch Completion                                     0
Culmen Length (mm)                                    0
Culmen Dept

In [172]:
sex_features = ['Culmen Length (mm)', 'Culmen Depth (mm)',
       'Flipper Length (mm)','Species_Adelie Penguin (Pygoscelis adeliae)',
       'Species_Chinstrap penguin (Pygoscelis antarctica)',
       'Species_Gentoo penguin (Pygoscelis papua)', 'Island_Biscoe', 'Island_Dream', 'Island_Torgersen'
       ]

models = []
models.append(('LR',LogisticRegression(solver='liblinear',multi_class = 'ovr')))
models.append(('LDA',LinearDiscriminantAnalysis()))
models.append(('KNN',KNeighborsClassifier()))
models.append(('CART',DecisionTreeClassifier()))
models.append(('NB',GaussianNB()))
models.append(('SVM',SVC(gamma='auto')))
models.append(('RFC',RandomForestClassifier()))
models.append(('XGBC',XGBClassifier(iterations=10000,verbose=False)))
models.append(('LGBMC',LGBMClassifier()))
models.append(('AdaC',AdaBoostClassifier()))
results =[]
names = []
for name, model in models:
  kfold = KFold(n_splits=10,random_state=7,shuffle = True)
  cv_results = cross_val_score(model,all[sex_features].iloc[all['Sex'].dropna().index]
                               ,all['Sex'].iloc[all['Sex'].dropna().index]
                               ,cv= kfold,scoring='accuracy')
  results.append(cv_results)
  names.append(name)
  msg = "%s : %f (%f) "%(name,cv_results.mean(),cv_results.std())
  print(msg)

LR : 0.781439 (0.066829) 
LDA : 0.870265 (0.058657) 
KNN : 0.833144 (0.044832) 
CART : 0.947727 (0.052823) 
NB : 0.728030 (0.075169) 
SVM : 0.889205 (0.045462) 
RFC : 0.956913 (0.031397) 
XGBC : 0.913636 (0.042527) 
LGBMC : 0.947727 (0.033378) 
AdaC : 0.895170 (0.033816) 


In [173]:
sex_model = RandomForestClassifier()
sex_model.fit(all[sex_features].iloc[all['Sex'].dropna().index],all['Sex'].iloc[all['Sex'].dropna().index])
train['Sex'].iloc[np.where(train['Sex'].isnull()==True)] =  sex_model.predict(train[train['Sex'].isnull()][sex_features])
test['Sex'].iloc[np.where(test['Sex'].isnull()==True)] =  sex_model.predict(test[test['Sex'].isnull()][sex_features])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [174]:
Delta_features = ['Culmen Length (mm)', 'Culmen Depth (mm)',
       'Flipper Length (mm)','Species_Adelie Penguin (Pygoscelis adeliae)',
       'Species_Chinstrap penguin (Pygoscelis antarctica)',
       'Species_Gentoo penguin (Pygoscelis papua)', 'Island_Biscoe', 'Island_Dream', 'Island_Torgersen','Sex'
       ]

In [175]:
models = []
models.append(('LR',LinearRegression()))
models.append(('LASSO',Lasso()))
models.append(('KNN',KNeighborsRegressor()))
models.append(('CART',DecisionTreeRegressor()))
models.append(('EN',ElasticNet()))
models.append(('SVM',SVR()))
models.append(('RFR',RandomForestRegressor()))
models.append(('XGBR',XGBRegressor(iterations=10000,verbose=False)))
models.append(('LGBMR',LGBMRegressor()))
models.append(('AdaR',AdaBoostRegressor()))

results =[]
names = []
for name, model in models:
  kfold = KFold(n_splits=10,random_state=7,shuffle = True)
  cv_results = cross_val_score(model,all[Delta_features].iloc[all['Delta 15 N (o/oo)'].dropna().index]
                               ,all['Delta 15 N (o/oo)'].iloc[all['Delta 15 N (o/oo)'].dropna().index]
                               ,cv= kfold,scoring='neg_mean_squared_error')
  results.append(cv_results)
  names.append(name)
  msg = "%s : %f (%f) "%(name,cv_results.mean(),cv_results.std())
  print(msg)

LR : -0.122081 (0.027126) 
LASSO : -0.241071 (0.039259) 
KNN : -0.134840 (0.029572) 
CART : -0.129978 (0.030921) 
EN : -0.220587 (0.037271) 
SVM : -0.214196 (0.034958) 
RFR : -0.084821 (0.017664) 
[13:51:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:51:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:51:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:51:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:51:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:51:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:51:42] WARNING: /workspace/src/object

In [176]:
d15_model = RandomForestRegressor()
d15_model.fit(all[Delta_features].iloc[all['Delta 15 N (o/oo)'].dropna().index]
                               ,all['Delta 15 N (o/oo)'].iloc[all['Delta 15 N (o/oo)'].dropna().index])
train['Delta 15 N (o/oo)'].iloc[np.where(train['Delta 15 N (o/oo)'].isnull()==True)] =  d15_model.predict(train[train['Delta 15 N (o/oo)'].isnull()][Delta_features])
test['Delta 15 N (o/oo)'].iloc[np.where(test['Delta 15 N (o/oo)'].isnull()==True)] =  d15_model.predict(test[test['Delta 15 N (o/oo)'].isnull()][Delta_features])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [177]:
models = []
models.append(('LR',LinearRegression()))
models.append(('LASSO',Lasso()))
models.append(('KNN',KNeighborsRegressor()))
models.append(('CART',DecisionTreeRegressor()))
models.append(('EN',ElasticNet()))
models.append(('SVM',SVR()))
models.append(('RFR',RandomForestRegressor()))
models.append(('XGBR',XGBRegressor(iterations=10000,verbose=False)))
models.append(('LGBMR',LGBMRegressor()))
models.append(('AdaR',AdaBoostRegressor()))
results =[]
names = []
for name, model in models:
  kfold = KFold(n_splits=10,random_state=7,shuffle = True)
  cv_results = cross_val_score(model,all[Delta_features].iloc[all['Delta 13 C (o/oo)'].dropna().index]
                               ,all['Delta 13 C (o/oo)'].iloc[all['Delta 13 C (o/oo)'].dropna().index]
                               ,cv= kfold,scoring='neg_mean_squared_error')
  results.append(cv_results)
  names.append(name)
  msg = "%s : %f (%f) "%(name,cv_results.mean(),cv_results.std())
  print(msg)

LR : -0.242914 (0.077114) 
LASSO : -0.430813 (0.064457) 
KNN : -0.289427 (0.040959) 
CART : -0.165639 (0.079138) 
EN : -0.360661 (0.068328) 
SVM : -0.479987 (0.067457) 
RFR : -0.136701 (0.044755) 
[13:51:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:51:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:51:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:51:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:51:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:51:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:51:49] WARNING: /workspace/src/object

In [178]:
d13_model = RandomForestRegressor()
d13_model.fit(all[Delta_features].iloc[all['Delta 13 C (o/oo)'].dropna().index]
                               ,all['Delta 13 C (o/oo)'].iloc[all['Delta 13 C (o/oo)'].dropna().index])
train['Delta 13 C (o/oo)'].iloc[np.where(train['Delta 13 C (o/oo)'].isnull()==True)] =  d13_model.predict(train[train['Delta 13 C (o/oo)'].isnull()][Delta_features])
test['Delta 13 C (o/oo)'].iloc[np.where(test['Delta 13 C (o/oo)'].isnull()==True)] =  d13_model.predict(test[test['Delta 13 C (o/oo)'].isnull()][Delta_features])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [179]:
test=test.drop(columns='Body Mass (g)')
print(test)
train.to_csv('new_train.csv',index=False)
test.to_csv('new_test.csv',index=False)

     Clutch Completion  ...  Species_Gentoo penguin (Pygoscelis papua)
0                    1  ...                                          0
1                    1  ...                                          1
2                    1  ...                                          0
3                    1  ...                                          0
4                    0  ...                                          0
..                 ...  ...                                        ...
223                  1  ...                                          0
224                  1  ...                                          1
225                  1  ...                                          1
226                  1  ...                                          0
227                  1  ...                                          0

[228 rows x 13 columns]


In [180]:
num_folds= 10
seed = 7
scoring = 'neg_root_mean_squared_error'

X_all = train[test.columns.tolist()]
y_all =train['Body Mass (g)']

In [181]:
X_train, X_valid, y_train, y_valid = train_test_split(train[test.columns.tolist()],train['Body Mass (g)']
                                                      ,test_size=0.2,random_state=seed)

In [182]:
models = []
models.append(('LR',LinearRegression()))
models.append(('LASSO',Lasso()))
models.append(('KNN',KNeighborsRegressor()))
models.append(('CART',DecisionTreeRegressor()))
models.append(('EN',ElasticNet()))
models.append(('SVM',SVR()))
models.append(('RFR',RandomForestRegressor()))
models.append(('XGBR',XGBRegressor()))
models.append(('LGBMR',LGBMRegressor()))
models.append(('AdaR',AdaBoostRegressor()))
models.append(('Xtree',ExtraTreesRegressor()))

results =[]
names = []
for name, model in models:
  kfold = KFold(n_splits=10,random_state=7,shuffle = True)
  cv_results = cross_val_score(model,X_train,y_train
                               ,cv= kfold,scoring=scoring)
  results.append(cv_results)
  names.append(name)
  msg = "%s : %f (%f) "%(name,cv_results.mean(),cv_results.std())
  print(msg)

LR : -329.250497 (55.796625) 
LASSO : -327.811972 (54.513088) 
KNN : -388.766440 (54.708975) 
CART : -419.103774 (58.433948) 
EN : -376.419383 (57.038744) 
SVM : -775.629957 (193.158685) 
RFR : -336.946026 (97.161290) 
[13:52:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:03] WARNING:

In [183]:
pipelines = []
pipelines.append(('ScaledLR',Pipeline([('Scaler',preprocessing.StandardScaler()),('LR',LinearRegression())])))
pipelines.append(('ScaledLASSO',Pipeline([('Scaler',preprocessing.StandardScaler()),('LASSO',Lasso())])))
pipelines.append(('ScaledKNN',Pipeline([('Scaler',preprocessing.StandardScaler()),('KNN',KNeighborsRegressor())])))
pipelines.append(('ScaledCART',Pipeline([('Scaler',preprocessing.StandardScaler()),('CART',DecisionTreeRegressor())])))
pipelines.append(('ScaledEN',Pipeline([('Scaler',preprocessing.StandardScaler()),('EN',ElasticNet())])))
pipelines.append(('ScaledSVM',Pipeline([('Scaler',preprocessing.StandardScaler()),('SVM',SVR())])))
pipelines.append(('ScaledRFR',Pipeline([('Scaler',preprocessing.StandardScaler()),('RFR',RandomForestRegressor())])))
pipelines.append(('ScaledXGBR',Pipeline([('Scaler',preprocessing.StandardScaler()),('XGBR',XGBRegressor())])))
pipelines.append(('ScaledLGBMR',Pipeline([('Scaler',preprocessing.StandardScaler()),('LGBMR',LGBMRegressor())])))
pipelines.append(('ScaledAdaR',Pipeline([('Scaler',preprocessing.StandardScaler()),('AdaR',AdaBoostRegressor())])))
pipelines.append(('ScaledXtree',Pipeline([('Scaler',preprocessing.StandardScaler()),('Xtree',ExtraTreesRegressor())])))

results_scaled =[]
names_scaled = []
for name, model in pipelines:
  kfold = KFold(n_splits=10,random_state=7,shuffle = True)
  cv_results = cross_val_score(model,X_train,y_train
                               ,cv= kfold,scoring=scoring)
  results_scaled.append(cv_results)
  names_scaled.append(name)
  msg = "%s : %f (%f) "%(name,cv_results.mean(),cv_results.std())
  print(msg)

ScaledLR : -329.250497 (55.796625) 
ScaledLASSO : -328.978433 (54.498665) 
ScaledKNN : -343.609297 (70.157249) 
ScaledCART : -443.544597 (73.570612) 
ScaledEN : -343.513482 (60.269369) 
ScaledSVM : -767.478981 (192.286413) 
ScaledRFR : -333.450779 (75.157569) 
[13:52:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favo

In [184]:
X_test=test

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_all)
X_all=scaler.transform(X_all)
X_test=scaler.transform(X_test)


from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(max_depth=5, n_estimators=200, min_samples_leaf=8, min_samples_split=8)
model.fit(X_all,y_all)

from sklearn.linear_model import Lasso
model_la=Lasso(alpha=0.2)
model_la.fit(X_all,y_all)

from sklearn.linear_model import LinearRegression
model_lr=LinearRegression()
model_lr.fit(X_all,y_all)

import xgboost as xgb
from xgboost import XGBRegressor
model_xgb=XGBRegressor(max_depth=2)
model_xgb.fit(X_all,y_all)

model_xt=ExtraTreesRegressor()
model_xt.fit(X_all,y_all)

pred=model.predict(X_test)
pred2=model_la.predict(X_test)
pred3=model_lr.predict(X_test)
pred4=model_xgb.predict(X_test)
pred5=model_xt.predict(X_test)

pred_fin=0.2*pred+0.2*pred2+0.2*pred3+0.2*pred4+0.2*pred5
pred_fin=pd.DataFrame(pred_fin)
pred_fin.columns=['Body Mass (g)']

sub=pd.concat([test_id,pred_fin], axis=1)
sub.columns=['id','Body Mass (g)']
print(sub)

sub.to_csv('penguin_massregress.csv', index=False)

[13:52:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
      id  Body Mass (g)
0      0    4299.072957
1      1    5618.392469
2      2    3552.990148
3      3    3406.797551
4      4    3485.140718
..   ...            ...
223  223    4146.452126
224  224    4765.550244
225  225    4612.496291
226  226    3910.193105
227  227    4114.112974

[228 rows x 2 columns]
